In [10]:
# Dynamic Pricing for Urban Parking Lots: Models 1 & 2 


# SETUP AND IMPORT

!pip install bokeh

import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
from bokeh.io import push_notebook
output_notebook()


# LOAD DATA

# Upload dataset.csv manually in Colab
# from google.colab import files
# uploaded = files.upload()

# Load the CSV file
# filename = list(uploaded.keys())[0]
df = pd.read_csv(r"C:\Users\User\Desktop\SA25\CapstoneProject\dataset.csv")


# PREPROCESSING

df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

vehicle_type_map = {"bike": 0.5, "car": 1.0, "truck": 1.5}
traffic_map = {"low": 0, "medium": 1, "high": 2}

df["VehicleTypeWeight"] = df["VehicleType"].map(vehicle_type_map)
df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map)

df.drop(columns=["ID", "LastUpdatedDate", "LastUpdatedTime", "VehicleType", "TrafficConditionNearby"], inplace=True)
df.sort_values(by="Timestamp", inplace=True)
df_clean = df.dropna(subset=["VehicleTypeWeight"])


# MODEL 1: BASELINE LINEAR MODEL

BASE_PRICE = 10
ALPHA = 2.0
df_clean["Price_Model1"] = np.nan

for lot_id, group in df_clean.groupby("SystemCodeNumber"):
    group = group.sort_values("Timestamp")
    prices = [BASE_PRICE]
    for i in range(1, len(group)):
        prev_price = prices[-1]
        occupancy = group.iloc[i]["Occupancy"]
        capacity = group.iloc[i]["Capacity"]
        price = BASE_PRICE + ALPHA * (occupancy / capacity) 

        prices.append(price)
    df_clean.loc[group.index, "Price_Model1"] = prices


# MODEL 2: DEMAND-BASED PRICING

ALPHA = 1.0
BETA = 0.3
GAMMA = 0.5
DELTA = 1.0
EPSILON = 0.5
LAMBDA = 0.5

# Compute demand
BASE_PRICE = 10
df_clean["DemandScore"] = (
    ALPHA * (df_clean["Occupancy"] / df_clean["Capacity"]) +
    BETA * df_clean["QueueLength"] -
    GAMMA * df_clean["TrafficLevel"] +
    DELTA * df_clean["IsSpecialDay"] +
    EPSILON * df_clean["VehicleTypeWeight"]
)

# Normalize demand per timestamp
df_clean["NormalizedDemand"] = df_clean.groupby("Timestamp")["DemandScore"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)

# Final price
df_clean["Price_Model2"] = BASE_PRICE * (1 + LAMBDA * df_clean["NormalizedDemand"])
df_clean["Price_Model2"] = df_clean["Price_Model2"].clip(lower=5, upper=20)


# VISUALIZATION

# Pick a sample lot
sample_lot = df_clean["SystemCodeNumber"].iloc[0]
sample_df = df_clean[df_clean["SystemCodeNumber"] == sample_lot]

p = figure(title=f"Pricing Models Over Time for Lot: {sample_lot}",
           x_axis_type="datetime",
           x_axis_label="Timestamp",
           y_axis_label="Price ($)",
           width=1200,
           height=600
        
)

# Model 1
p.line(sample_df["Timestamp"], sample_df["Price_Model1"],
       legend_label="Model 1: Linear",
       line_color="blue", line_width=2)

# Model 2
p.line(sample_df["Timestamp"], sample_df["Price_Model2"],
       legend_label="Model 2: Demand-Based",
       line_color="green", line_width=2)

# Hover tool
hover = HoverTool(tooltips=[
    ("Time", "@x{%F %H:%M}"),
    ("Price", "@y{$0.00}")
], formatters={"@x": "datetime"}, mode="vline")

p.add_tools(hover)
p.legend.location = "top_left"
show(p)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Loading BokehJS ...

C:\Users\User\AppData\Local\Temp\ipykernel_11856\2773810713.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Price_Model1"] = np.nan
C:\Users\User\AppData\Local\Temp\ipykernel_11856\2773810713.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["DemandScore"] = (
C:\Users\User\AppData\Local\Temp\ipykernel_11856\2773810713.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 